In [8]:
import pyvisa
from qcodes.instrument_drivers.rohde_schwarz import (
    RohdeSchwarzZNB8,
    RohdeSchwarzZNBChannel,
)
import matplotlib.pyplot as plt
import matplotlib
%matplotlib qt

import numpy as np
from numpy import *
from scipy.fft import fft, ifft, fftshift
from scipy.io import savemat
from datetime import datetime
import time

import os
Global_data_adress = r"C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA"
exp_name = 'TWPA_noise'
adress = Global_data_adress+'\\'+exp_name
if not os.path.exists(adress):
  os.mkdir(adress)
  print("Folder %s created!" % adress)
else:
  print("Folder %s already exists" % adress)

Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise already exists


Create a folder for measurement data

In [9]:
now = datetime.now()
path = adress +'\\'+ now.strftime('%Y-%m-%d')
if not os.path.exists(path):
  os.mkdir(path)
  print("Folder %s created!" % path)
else:
  print("Folder %s already exists" % path)

Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise\2023-09-19 already exists


# VNA init

In [10]:
def magtodb(mag): return 20*np.log10(mag)

def vna_setup_CW(cw_frequency=6e9, vna_power=-30, bandwidth=10):
    try: print(vna.query('*IDN?'))                 # check if connection already exist
    except:                                        # if no connection
        rm = pyvisa.ResourceManager()
        vna = rm.open_resource('GPIB0::20::INSTR') # GPIB connection
        print(vna.query('*IDN?'))
        
    vna.write(':SENS1:BAND {}'.format(str(bandwidth))) # set defoult bandwidth
    vna.write('SOUR1:POW {}'.format(str(vna_power)))   # set defoult power
    vna.write('INIT1:CONT OFF')                        # single-sweep mode
    vna.write('SENS1:SWE:POIN 201')                    # set maximum possible number of points for single sweep
    vna.write(':SENS1:SWE:TYPE CW')                    # set CW mode of measurements
    return vna


def vna_on(): vna.write('OUTP ON')
def vna_off(): vna.write('OUTP OFF')
    
def vna_meas_points(): 
    return int(vna.query('SENS1:SWE:POIN?'))  # should be 201

def vna_sweep_time(): 
    return float(vna.query('SENS1:SWE:TIME?'))  # estimated sweep time for current settings of vna

def vna_set_cw_freq(freq): 
    vna.write('SENS1:FREQ:CW {}'.format(str(freq))) # set single frequency point for CW mode in Hz

def vna_set_power(power): 
    vna.write('SOUR1:POW {}'.format(str(power))) # set power
    
def vna_set_band(bandwidth): 
    vna.write(':SENS1:BAND {}'.format(str(bandwidth))) # set power
    
def vna_meas_cw():
    vna_on()
    vna.write("INIT1:IMM")                              # start measurement
    time.sleep(0.2+float(vna.query('SENS1:SWE:TIME?'))) # this part is important becouse the measurement nedds to finnish before readout process
    vna.write("CALC1:FORM MLOG")                        # format of data
    data_str = vna.query("CALC1:DATA? SDAT")            # readout
    
    data = np.array(data_str.rstrip().split(",")).astype("float64")
    s = data[0::2] + 1j * data[1::2]
    i = data[0::2]
    q = data[1::2]
    return abs(s), angle(s)

In [11]:
vna = vna_setup_CW() # initializing or checking for connection

Rohde-Schwarz,ZNB20-2Port,1311601062102129,3.45



# Single time sweep

In [168]:
# Check time domain data
vna_set_cw_freq(6e9)
vna_set_power(-30)
anapico_power(-20)

mag, pha = vna_meas_cw()
print('number of points measured {} of {}'.format(len(pha),int(vna_meas_points())))

fig, ax = plt.subplots(1,2,figsize=(15,7), sharex=True)
ax[0].plot(fft_f(), mag)
# ax[0].plot(fft_f(), magtodb(mag))
ax[1].plot(fft_f(), pha)
ax[0].set_xlabel('Frequency, GHz')
ax[1].set_xlabel('Frequency, GHz')
ax[0].set_ylabel('Magnitude')
ax[1].set_ylabel('Phase')

number of points measured 201 of 201


Text(0, 0.5, 'Phase')

# ANAPICO init

In [12]:
def setup_anapico(power=-30, frequency = 6e9, channel = 1):
    try: print(anapico.query('*IDN?'))
    except: 
        rm = pyvisa.ResourceManager()
#         anapico = rm.open_resource('USB0::0x03EB::0xAFFF::3C6-0B4F40003-0985::INSTR') # USB connection
        anapico = rm.open_resource('TCPIP0::169.254.5.91::inst0::INSTR') # LAN connection
        print(anapico.query('*IDN?'))
    if power<=-6:
#         print('Flux power set:', power)
        anapico.write(r'SOUR{}:POW {}'.format(str(channel), str(power)))
        anapico.write(r'SOUR{}:FREQ {}'.format(str(channel), str(frequency)))
    else: 
        print('Flux power is too high:', power)
    return anapico

def anapico_on():
    anapico.write(r'OUTP1 ON')

def anapico_off():
    anapico.write(r'OUTP1 OFF')

def anapico_freq(freq, channel=1): 
    anapico.write(r'SOUR{}:FREQ {}'.format(str(channel), str(freq)))

def anapico_power(power, channel=1): 
    anapico.write(r'SOUR{}:POW {}'.format(str(channel), str(power)))

In [15]:
anapico = setup_anapico() # initializing or checking for connection

AnaPico AG,APMS20G,3C6-0B4F40003-0985,0.4.204



In [17]:
anapico.close()

# Single frequency flux power sweep

In [179]:
#_____________ANAPICO______________
flux_powers = np.linspace(-25, -6, 5)
flux_freq = 11.252e9 # maximum gain frequency

#_____________VNA______________
vna_freq = 5.78195e9  #resonance frequency
vna_bandwidth = 10
number_of_points = vna_meas_points()
vna_power = 0

avg_total = 2  #separate measurements of vna per each flux power

In [180]:
def fft_A(A):
    fft_A = abs(fftshift(fft(A))**2)
    return fft_A

def fft_f(bandwidth=vna_bandwidth, nop=number_of_points):
    f = np.linspace(-bandwidth/2, bandwidth/2, nop)
    return f

In [181]:
anapico_power(-15)
anapico_freq(flux_freq)
anapico_on()

vna_set_power(vna_power)


now1 = datetime.now()
data = dict.fromkeys(flux_powers, None)

for ip, flux_power in enumerate(flux_powers):
    print('round {}\{}, current flux power = {}'.format(ip, len(flux_powers), flux_power))
    
    anapico_power(flux_power)
    
    fft_avg_mag = np.zeros(number_of_points)
    fft_avg_pha = np.zeros(number_of_points)
    for avg in range(avg_total):
        mag, pha = vna_meas_cw()
        fft_avg_mag+=fft_A(mag)
        fft_avg_pha+=fft_A(pha)

    data[flux_power] = np.array([[fft_avg_mag/avg_total], [fft_avg_pha/avg_total]])

now2 = datetime.now()
anapico_off()
print('Run time:', now2-now1)
# anapico.close()

InvalidSession: Invalid session handle. The resource might be closed.

In [105]:
# %matplotlib qt
for i in range(len(flux_powers)):
    plt.loglog(fft_f(), data[flux_powers[i]][1][0])

In [106]:
data_to_save=np.empty((len(flux_powers),2,number_of_points))
for i in range(len(flux_powers)):
    for j in range(2):
        for k in range(number_of_points): 
            data_to_save[i,j,k] = data[flux_powers[i]][j][0][k]

In [107]:
dict_data=dict()
dict_data['flux_powers'] = flux_powers
dict_data['data'] = data_to_save
dict_data['freqs'] = fft_f()

experiment_name = 'Flux_pow_sweep'
file_name = path+'\\'+experiment_name+now.strftime(r'%H-%M-%S')+'.mat'
savemat(file_name, dict_data)

In [117]:
ind = [0, 4,  8, 9]
fig, ax = plt.subplots(1,2,figsize=(15,7), sharex=True, sharey=True)
for i in range(len(flux_powers)):
    ax[0].loglog(fft_f(), fft_A(data[flux_powers[i]][1][0]), label = '{}'.format(round(flux_powers[i],2)))
    ax[1].loglog(fft_f(), fft_A(data[flux_powers[i]][0][0]), label = '{}'.format(round(flux_powers[i],2)))
    
ax[0].set_xlabel('Frequency, GHz')
ax[1].set_xlabel('Frequency, GHz')
ax[0].set_ylabel('Magnitude')
ax[1].set_ylabel('Phase')
ax[0].legend()
ax[1].legend()

KeyError: -18.0

In [58]:
anapico.close() # before restarting kernel connection to anapico should be closed. vna is fine.

# Flux Frequency+Power sweep

In [175]:
#_____________ANAPICO______________
flux_powers = np.linspace(-30, -6, 5)
flux_freqs = np.linspace(11.252e9-2000e6, 11.252e9+2000e6, 5)  # maximum gain at 11.252e9

#_____________VNA______________
vna_freq = 5.78195e9  #resonance frequency
vna_bandwidth = 10
number_of_points = vna_meas_points()
vna_power = 0

avg_total = 3  #separate measurements of vna per each flux power

In [176]:
vna_set_power(vna_power)
vna_set_band(vna_bandwidth)
vna_set_cw_freq(vna_freq)

data_to_save=np.empty((len(flux_freqs), len(flux_powers), 2, number_of_points))
data_to_save_fft=np.empty((len(flux_freqs), len(flux_powers), 2, number_of_points))

now1 = datetime.now()
print('Start: {}'.format(now.strftime(r'%H-%M-%S')))
for i_fr, flux_freq in enumerate(flux_freqs):
    print('round {}\{}, current flux freq = {}'.format(i_fr, len(flux_freqs), round(flux_freq/1e9,2)))
    anapico_freq(flux_freq)
    anapico_on()

    for j_fl, flux_power in enumerate(flux_powers):
        print('round {}\{}, current flux power = {}'.format(j_fl, len(flux_powers), flux_power))
        anapico_power(flux_power)

        avg_mag = np.zeros(number_of_points)
        avg_pha = np.zeros(number_of_points)
        avg_mag_fft = np.zeros(number_of_points)
        avg_pha_fft = np.zeros(number_of_points)
        for avg in range(avg_total):
            try: mag, pha = vna_meas_cw()
            except: 
                vna = vna_setup_CW(cw_frequency=vna_freq, vna_power=vna_power, bandwidth=vna_bandwidth)
                mag, pha = vna_meas_cw()
                
            avg_mag_fft+=fft_A(mag)
            avg_pha_fft+=fft_A(pha)
            
            avg_mag+=mag
            avg_pha+=pha
            
        tot_mag = avg_mag/avg_total
        tot_pha = avg_pha/avg_total
        
        tot_mag_fft = avg_mag_fft/avg_total
        tot_pha_fft = avg_pha_fft/avg_total
        
        for i in range(len(tot_mag)): 
            data_to_save[i_fr,j_fl,0,i] = tot_mag[i] # magnitude
            data_to_save[i_fr,j_fl,1,i] = tot_pha[i] # phase
            
            data_to_save_fft[i_fr,j_fl,0,i] = tot_mag_fft[i] # magnitude
            data_to_save_fft[i_fr,j_fl,1,i] = tot_pha_fft[i] # phase
    
anapico_off()
now2 = datetime.now()
print('Run time:', now2-now1)

round 0\5, current flux freq = 9.25
round 0\5, current flux power = -30.0
round 1\5, current flux power = -24.0
round 2\5, current flux power = -18.0
round 3\5, current flux power = -12.0
round 4\5, current flux power = -6.0
round 1\5, current flux freq = 10.25
round 0\5, current flux power = -30.0
round 1\5, current flux power = -24.0
round 2\5, current flux power = -18.0
round 3\5, current flux power = -12.0
round 4\5, current flux power = -6.0
round 2\5, current flux freq = 11.25
round 0\5, current flux power = -30.0
round 1\5, current flux power = -24.0
round 2\5, current flux power = -18.0
round 3\5, current flux power = -12.0
round 4\5, current flux power = -6.0
round 3\5, current flux freq = 12.25
round 0\5, current flux power = -30.0
round 1\5, current flux power = -24.0
round 2\5, current flux power = -18.0
round 3\5, current flux power = -12.0
round 4\5, current flux power = -6.0
round 4\5, current flux freq = 13.25
round 0\5, current flux power = -30.0
round 1\5, current flu

In [177]:
for i_fr, flux_freq in enumerate(flux_freqs[:15]):
    if i_fr%1==0:
        fig, ax = plt.subplots(1,2,figsize=(15,7), sharex=True, sharey=True)
        st = fig.suptitle("TWPA freq {}".format(round(flux_freq/1e9,5)), fontsize="x-large")

        for j_fl, flux_power in enumerate(flux_powers):
            if j_fl%1==0 and i%1==0:
                ax[0].loglog(fft_f(), data_to_save_fft[i_fr,j_fl,0,:], label = '{}'.format(round(flux_powers[j_fl],2)))
                ax[1].loglog(fft_f(), data_to_save_fft[i_fr,j_fl,1,:], label = '{}'.format(round(flux_powers[j_fl],2)))

                ax[0].set_xlabel('Frequency, GHz')
                ax[1].set_xlabel('Frequency, GHz')
                ax[0].set_ylabel('Magnitude')
                ax[1].set_ylabel('Phase')
                ax[0].legend()
                ax[1].legend()

In [173]:
dict_data=dict()
dict_data['flux_powers'] = flux_powers
dict_data['flux_freqs'] = flux_freqs
dict_data['vna_freq'] = vna_freq
dict_data['vna_bandwidth'] = vna_bandwidth
dict_data['number_of_points'] = number_of_points
dict_data['vna_power'] = vna_power
dict_data['avg_total'] = avg_total

dict_data['data'] = data_to_save
dict_data['data_fft'] = data_to_save_fft


experiment_name = 'Freq_Flux_pow_sweep'
file_name = path+'\\'+experiment_name+now.strftime(r'%H-%M-%S')+'.mat'
savemat(file_name, dict_data)

Create log file

In [174]:
comment = 'Freq+Power Sweep - ok. in resonance high probe'


file_name = adress +'\\'+'log_'+exp_name+'.txt'
f = open(file_name, 'a')  # append  to existing file
f.write(str(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))+':   ')
f.write(comment+'\n')
f.close()